<h1>This is the blue print for all transfer learning techniques</h1>

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# transfer learning is 1001 classes classification from 0 to 1000
# they are trained on 1.4M images of size (224,224)
image_size = np.array([224, 224])

In [6]:
image_size

array([224, 224])

In [70]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# to run the below code we get errors so we write above code

In [71]:
# include_top=False removes the last dense layer which contain 1001 neurons
# weights parameter will import the weights of vgg16 model which is already trained
vgg = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [72]:
# we no need to train the existing weights so we are making trainable parameter to False
for layer in vgg.layers:
  layer.trainable = False

In [73]:
# we are flattening the last layer output
# after passing the images to all layers the last layer output we get we flatten it
# suppose the last layer output is (9, 9, 2) after flattening we get 168 
x = Flatten()(vgg.output)

In [74]:
# after flattening we got 25088 as output
# None means for every input of image we get 25088
x

<KerasTensor: shape=(None, 25088) dtype=float32 (created by layer 'flatten_1')>

In [75]:
# after flattening the vgg16 layers we now add last layer with 5neurons for prediction
## by freezing all the layers of vgg16
prediction = Dense(5, activation='softmax')(x)

In [76]:
prediction

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense_1')>

In [77]:
# now finally we are creating model 
## by combining all flatten layers and prediction layer
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
# from below we can see that trainable parameters are only 125445

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [78]:
# now we should tell the model what optimization and loss function should be used
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# we use only train_datagen because we are creating samples from only one folder flower_photos
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# directory should contain subdirectories of classes each contain jpg or png images
# target_size is the dimensions to which all images found inside subdirectories will be resized.
# classes is the all subdirectories 
# class_mode='sparse' will assign an integer to the classes eg[0, 1, 2, 3, 4]
## if class_mode is 'categorical' it will convert outputs to one_hot representation
# batch_size is size of batches of data
training_set = train_datagen.flow_from_directory(directory='datasets/train',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 classes=['daisy', 'dandelion', 'roses', 'sunflowers',
                                                          'tulips'],
                                                 class_mode='sparse')


Found 3490 images belonging to 5 classes.


In [59]:
# if we are not using train_test_split then we use below code 
## because training set is different and testing set is different we use below code
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('datasets/test',
                                            target_size=(224, 224),
                                            batch_size=32,
                                            classes=['daisy', 'dandelion', 'roses', 'sunflowers',
                                                     'tulips'],
                                            class_mode='sparse')
# we created test directory with 5 subdirectories
## each subdirectory contains 36 images
# so total images are 36*5=180

Found 180 images belonging to 5 classes.


In [60]:
# each element in test_set is tuple of 4d arrays and its corresponding ys
i = 0
for j in test_set:
    print(j[0].shape)
    i += 1
    if i>18:
        break
# by observing the below pattern there are 32 5s and one 20 this sequence will repeat 
## so total 32*5 + 20 = 180
# and the above pattern will repeat for infinite no of times so we are using break statement

(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(20, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(20, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(32, 224, 224, 3)
(20, 224, 224, 3)
(32, 224, 224, 3)


In [62]:
# each element in test_set is tuple of 4d arrays and its corresponding ys
for j in test_set:
    print(j[0].shape)
    print(j[1])
    print(len(j[1]))
    break
# by observing the below pattern there are 32 5s and one 20 this sequence will repeat 
## so total 32*5 + 20 = 180
# and the above pattern will repeat for infinite no of times so we are using break statement

(32, 224, 224, 3)
[4. 0. 0. 3. 4. 0. 1. 3. 4. 3. 0. 3. 3. 4. 4. 4. 2. 1. 3. 4. 0. 0. 2. 0.
 0. 3. 1. 1. 0. 1. 2. 3.]
32


In [66]:
type(test_set)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [64]:
len(test_set) # 32 5s and 20 one so total 6

6

In [79]:
# fit the model
ml = model.fit(training_set, validation_data=test_set, epochs=2,
                                                  steps_per_epoch=len(training_set),
                                                  validation_steps=len(test_set))



Epoch 1/2
110/110 [==============================] - 164s 1s/step - loss: 1.1549 - accuracy: 0.5840 - val_loss: 0.8295 - val_accuracy: 0.7444
Epoch 2/2
110/110 [==============================] - 147s 1s/step - loss: 0.4798 - accuracy: 0.8436 - val_loss: 0.8978 - val_accuracy: 0.7778


In [80]:
ml.history

{'loss': [0.870847225189209, 0.477873831987381],
 'accuracy': [0.6902579069137573, 0.8383954167366028],
 'val_loss': [0.8295403122901917, 0.8978334069252014],
 'val_accuracy': [0.7444444298744202, 0.7777777910232544]}